## SQL Project: Answering Business Questions  

### 1. Introduction and Schema Diagram

In this project, we are using the Chinook database,  is provided as a SQLite database file called `chinook.db`. A copy of the database schema is below.

![chinook.db](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

We are going to use SQL queries to analyse the following: 

* The best selling genre in USA
* Performance of the sales representative 
* Sales by country
* Purchase of albums vs purchase of individual tracks
* Which artist is used in the most playlists?
* How many tracks have been purchased vs not purchased?
* Is the range of tracks in the store reflective of their sales popularity?
* Do protected vs non-protected media types have an effect on popularity?

Let's connect Jupyter Notebook to the database file and get started with our analysis.

In [4]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### List of  tables and views in the database

In [5]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Recognizing the database

In [6]:
%%sql
SELECT DISTINCT *
FROM media_type
WHERE name LIKE '%protected%'

Done.


media_type_id,name
2,Protected AAC audio file
3,Protected MPEG-4 video file


In [7]:
%%sql
SELECT c.country,COUNT(*) AS 'Number _of_customers'
FROM customer AS c
GROUP BY c.country

Done.


country,Number _of_customers
Argentina,1
Australia,1
Austria,1
Belgium,1
Brazil,5
Canada,8
Chile,1
Czech Republic,2
Denmark,1
Finland,1


In [8]:
%%sql
SELECT c.first_name||' '||c.last_name AS 'customers_from_India'
FROM customer AS c
WHERE c.country='India'

Done.


customers_from_India
Manoj Pareek
Puja Srivastava


In [9]:
%%sql
SELECT DISTINCT quantity
FROM invoice_line

Done.


quantity
1


###  Genre and number of tracks sold in USA

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce. 

Let's query out which genres sell the best in the USA.


In [10]:
%%sql
WITH 
    invoice_usa AS 
    ( SELECT c.country,i.customer_id,i.invoice_id
      FROM customer AS c
      INNER JOIN invoice AS i
      ON c.customer_id=i.customer_id
      WHERE c.country="USA" 
    ),
    invoice_line_usa AS
    ( SELECT iu.country,il.*
      FROM invoice_usa AS iu
      INNER JOIN invoice_line AS il
      ON iu.invoice_id=il.invoice_id
    ),
    track_usa AS
    ( SELECT t.track_id,t.genre_id
     FROM invoice_line_usa AS ilu
     INNER JOIN track AS t
     ON ilu.track_id=t.track_id
    ),
    genre_usa AS
    ( SELECT *
     FROM track_usa AS tu
     INNER JOIN genre AS g
     ON tu.genre_id=g.genre_id
    )

 
SELECT name AS 'genre_name',
       COUNT(name) AS 'sold_in_usa',
       ROUND(COUNT(name)*100.0/(SELECT COUNT(*) FROM genre_usa),2) AS 'percentage'
FROM genre_usa AS gu
GROUP BY gu.genre_id
ORDER BY 2 DESC;


Done.


genre_name,sold_in_usa,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


## As the above query suggests
Top 3 genres sold in the USA are

* Rock
* Alternative & Punk
* Metal

And they make up nearly 78% of all sales.



### Analyzing performance of Sales Agents

In [11]:
%%sql
WITH 
    sales_agent AS
        (
            SELECT *
        FROM employee AS e
        WHERE e.title='Sales Support Agent'
        ),
     sales_agent_sales AS
        ( SELECT sa.*,c.customer_id
          FROM sales_agent AS sa
          INNER JOIN customer AS c
          ON sa.employee_id=c.support_rep_id
        ),
    sales_agent_invoices AS
        (
            SELECT sas.*,i.invoice_id,i.total
            FROM sales_agent_sales AS sas
            INNER JOIN invoice AS i
            ON sas.customer_id=i.customer_id
        )

SELECT first_name||' '||last_name AS 'name',
       birthdate,
       hire_date, 
       ROUND(SUM(total),2) AS 'Total Sales'
FROM sales_agent_invoices
GROUP BY employee_id

Done.


name,birthdate,hire_date,Total Sales
Jane Peacock,1973-08-29 00:00:00,2017-04-01 00:00:00,1731.51
Margaret Park,1947-09-19 00:00:00,2017-05-03 00:00:00,1584.0
Steve Johnson,1965-03-03 00:00:00,2017-10-17 00:00:00,1393.92


#### In the above query, we look into total sales of each  support agent.
As expected the sales amount is strongly correlated to the hire date of each agent.

###  Analyzing Sales by Country

In [17]:
%%sql
WITH 
  customer_invoice AS
    ( SELECT c.customer_id,c.country,i.invoice_id,i.total
     FROM customer AS c
     INNER JOIN invoice AS i
     ON c.customer_id=i.customer_id
    ),
    others AS
    ( SELECT country,
             COUNT(DISTINCT customer_id) AS 'number_of_customers',
             COUNT(invoice_id) AS 'number_of_invoices',
             ROUND(SUM(total),2) AS 'total'
             --ROUND(AVG(total),2)
      FROM customer_invoice AS ci
      GROUP BY country
      HAVING COUNT(DISTINCT customer_id)=1
    ) ,
    others_stat AS 
    (SELECT "Others" AS country,
             SUM(number_of_customers) AS 'number_of_customers',
             SUM(total) AS 'total_sales',
             ROUND(SUM(total)/SUM(number_of_customers),2) AS 'avg_sale_per_customer',
             ROUND(SUM(total)/SUM(number_of_invoices),2)  AS 'avg_order_value',
             2 AS 'rank'
     FROM others AS o
    ),
        main_countries AS
    ( SELECT country,
             COUNT(DISTINCT customer_id) AS 'number_of_customers',
             ROUND(SUM(total),2) AS 'total_sales',
             ROUND(SUM(total)/COUNT(DISTINCT customer_id),2) AS 'avg_sale_per_customer',
             ROUND(SUM(total)/COUNT(invoice_id),2)  AS 'avg_order_value',
             1 AS 'rank'
      FROM customer_invoice AS ci
      GROUP BY country
      HAVING COUNT(DISTINCT customer_id)>1
      ORDER BY 3
    ),
    
main_table AS
(
SELECT *

FROM main_countries

UNION

SELECT *

FROM others_stat

ORDER BY 6, 3 DESC)

SELECT
country,
number_of_customers,
total_sales,
avg_sale_per_customer,
avg_order_value
FROM main_table

Done.


country,number_of_customers,total_sales,avg_sale_per_customer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
Others,15,1094.94,73.0,7.45


### for each country :

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

are calculated. Here countries with only one customer are collected into an "Other" group.

### Which artist is used in the most playlists?

In [13]:
%%sql
WITH
playlist_artist AS
(SELECT pl.playlist_id,plt.track_id,ar.name
FROM playlist AS pl
INNER JOIN playlist_track AS plt
ON pl.playlist_id=plt.playlist_id
INNER JOIN track AS t
ON plt.track_id=t.track_id
INNER JOIN album AS a
ON t.album_id=a.album_id
INNER JOIN artist AS ar
ON a.artist_id=ar.artist_id)

SELECT name, count(name) AS 'Playlist_appearence'
FROM playlist_artist
GROUP BY name
ORDER BY 2 DESC
LIMIT 10;


Done.


name,Playlist_appearence
Iron Maiden,516
U2,333
Metallica,296
Led Zeppelin,252
Deep Purple,226
Lost,184
Pearl Jam,177
Eric Clapton,145
Faith No More,145
Lenny Kravitz,143


### How many tracks have been purchased vs not purchased?

In [14]:
%%sql
WITH
track_sold AS
(
SELECT t.track_id, count(il.invoice_line_id) AS 'times_sold'
FROM track AS t 
LEFT JOIN invoice_line AS il
ON il.track_id=t.track_id
GROUP BY 1
ORDER BY 2 DESC)

SELECT COUNT(*) AS 'Number_of_Tracks',
       (SELECT COUNT(*)
                      FROM track_sold
                      WHERE times_sold=0) AS 'not_purchased',
      (SELECT COUNT(*)
                      FROM track_sold
                      WHERE times_sold!=0) AS 'purchased'
FROM track_sold;

Done.


Number_of_Tracks,not_purchased,purchased
3503,1697,1806


### Is the length of tracks in the store reflective of their sales popularity?

In [15]:
%%sql
WITH
track_len AS
(SELECT track_id,
       ROUND(milliseconds/60000.0,1) AS 'length(sec)',
       CASE
            WHEN (milliseconds/60000.0)<3 THEN 'short'
            WHEN (milliseconds/60000.0)<5 THEN 'medium'
            ELSE 'long'
        END AS 'range'    
FROM track),

track_sales AS
(   SELECT tl.range, COUNT(il.track_id) AS 'sales'
    FROM track_len AS tl
    LEFT JOIN invoice_line AS il
    ON tl.track_id=il.track_id
    GROUP BY tl.range
)



SELECT *
FROM track_sales;



Done.


range,sales
long,1023
medium,2936
short,798


From the table above, one easily can conclude that medium length tracks are more popular.

### Do  media types have an effect on popularity?

In [22]:
%%sql

WITH
track_type AS
(
SELECT  t.track_id,mt.name
FROM track AS t
INNER JOIN media_type AS mt
ON t.media_type_id=mt.media_type_id
),

track_type_numbers AS
(
    SELECT name,COUNT(track_id) AS 'track'
    FROM track_type AS tt
    GROUP BY name
),

track_sales_protected AS
(   SELECT tt.name,
    COUNT(il.invoice_line_id) AS 'sales'
    FROM track_type AS tt
    LEFT JOIN invoice_line AS il
    ON tt.track_id=il.track_id
    GROUP BY tt.name
)

SELECT 
    ttn.name AS 'media_type',
    ttn.track AS 'number_of_tracks',
    tsp.sales AS 'number_of_sales',
    ROUND(CAST(tsp.sales AS float)/ttn.track,2) AS 'sales_media_type_ratio'
FROM track_type_numbers AS ttn
INNER JOIN track_sales_protected AS tsp
ON ttn.name=tsp.name

Done.


media_type,number_of_tracks,number_of_sales,sales_media_type_ratio
AAC audio file,11,21,1.91
MPEG audio file,3034,4259,1.4
Protected AAC audio file,237,439,1.85
Protected MPEG-4 video file,214,3,0.01
Purchased AAC audio file,7,35,5.0


Protected MPEG-4 video file media category has the worst sale statistics
and Purchased AAC audio file media type has best sale records.
We recommend the company reduce former track types and offer more the latter kind of tracks.